In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
import albumentations as A

from model.FoodDataset import FoodDataset

torch.cuda.empty_cache()
torch.cuda.memory_summary()

In [2]:
image_paths = ["E:\Licenta_DOC\API_Segmentation\data\generated\img_dir"]
seg_paths = ["E:\Licenta_DOC\API_Segmentation\data\generated\\ann_dir"]

LOAD_MODEL = False
def get_images(transform = None,batch_size=32,shuffle=True,pin_memory=True):
    data = FoodDataset(image_paths, seg_paths,transform = transform)
    train_size = int(0.8 * data.__len__())
    test_size = data.__len__() - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])
    train_batch = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, pin_memory=pin_memory)
    test_batch = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle, pin_memory=pin_memory)
    return train_batch,test_batch


In [3]:

from service.Service import Service

t1 = A.Compose([
    A.Resize(128, 128),
    A.augmentations.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

train_batch,test_batch = get_images(transform =t1, batch_size=8)

service = Service()

In [ ]:
for img,mask in train_batch:
    img1 = np.transpose(img[0,:,:,:],(1,2,0))
    mask1 = np.array(mask[0,:,:])
    img2 = np.transpose(img[1,:,:,:],(1,2,0))
    mask2 = np.array(mask[1,:,:])
    img3 = np.transpose(img[2,:,:,:],(1,2,0))
    mask3 = np.array(mask[2,:,:])
    fig , ax =  plt.subplots(2, 2, figsize=(18, 18))
    ax[0][0].imshow(img1)
    ax[0][1].imshow(mask1)
    print(service.get_bounding_boxes(mask1))
    break

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [5]:
import sys
sys.path.append("E:\PythonModels\segment-anything")
from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator

In [6]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))

In [ ]:
from PIL import Image

image = np.array(Image.open('E:\PythonModels\segment-anything\\notebooks\images\\truck.jpg'))
plt.figure(figsize=(10,10))
plt.imshow(image)
plt.axis('on')
plt.show()

In [ ]:
# sam_checkpoint = "checkpoints/sam_vit_h_4b8939.pth"
# model_type = "vit_h"

sam_checkpoint = "checkpoints/sam_vit_b_01ec64.pth"
model_type = "vit_b"

device = "cuda" if torch.cuda.is_available() else "cpu"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)

# predictor = SamPredictor(sam)
print(sam.image_encoder)


In [ ]:
from torchsummary import summary
model = sam.image_encoder.to(device)
print(summary(model, (3, 1024, 1024)))
print(sam)


In [7]:
layers = { name: module for name,module in sam.named_children()}
image_encoder = layers.get('image_encoder').to('cuda')
print(image_encoder)

ImageEncoderViT(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): MLPBlock(
        (lin1): Linear(in_features=768, out_features=3072, bias=True)
        (lin2): Linear(in_features=3072, out_features=768, bias=True)
        (act): GELU(approximate='none')
      )
    )
  )
  (neck): Sequential(
    (0): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): LayerNorm2d()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): LayerNorm2d()
  )
)


In [8]:
sam_image_encoder = sam.image_encoder.to(device)

In [ ]:
input = torch.randn(1, 3, 1024, 1024).to(device)

x = sam_image_encoder(input)


In [13]:
# without_neck = { name: module for name,module in image_encoder.named_children() if name != 'neck'}
# print(without_neck)
# class CustomModel(torch.nn.Module) :
#     def __init__(self, patch_embed, blocks):
#         super(CustomModel, self).__init__()
#         self.patch_embed = patch_embed
#         self.blocks = blocks
#
#     def forward(self, x):
#         x = self.patch_embed(x)
#         x = self.blocks(x)
#         return x
#
#
# model = CustomModel(**without_neck)

In [ ]:
from torchsummary import summary

print(summary(sam.image_encoder, input_size=(3, 1024, 1024)))
# image_encoder(torch.rand(1, 3 , 1024, 1024).to(device))

In [25]:
predictor.set_image(image)


In [ ]:
input_point = np.array([[500, 375]])
input_label = np.array([1])
plt.figure(figsize=(10,10))
plt.imshow(image)
show_points(input_point, input_label, plt.gca())
plt.axis('on')
plt.show()

In [ ]:
from model.checkpoints.checkpoints import load_checkpoint
from model.architectures.UnetRestNet50 import UNetResNet152
service = Service()
MODEL_PRETRAINED_FILE = 'checkpoints/checkpoint-pretrain.pth.tar'
model = UNetResNet152(104).to(DEVICE)
load_checkpoint(torch.load(MODEL_PRETRAINED_FILE), model)

In [ ]:
from torch import nn

image = Image.open('E:\Licenta_DOC\API_Segmentation\data\generated\img_dir\\00000000_aug_0.jpg')
x = np.array(image)
image = image.resize((256,256))
image = np.array(image)
t1 = A.Compose([
    A.Resize(256, 256),
    A.augmentations.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])
x = t1(image=x)['image']
x = x.to(DEVICE)
x = x.unsqueeze(0)
softmax = nn.Softmax(dim=1)
preds = torch.argmax(softmax(model(x)), axis=1).to('cpu')
preds1 = np.array(preds[0, :, :])


In [ ]:
boxes = service.get_bounding_boxes(preds1)


In [ ]:
from utils.category_reader import read_categories

CATEGORY_DICT_FILE = '../data/category_id.txt'
category_dict = read_categories(CATEGORY_DICT_FILE)
print(category_dict['48'])
print(category_dict['61'])
print(category_dict['47'])
print(category_dict['80'])

In [ ]:
predictor.set_image(np.array(image))

input_boxes = torch.tensor([[t_x, t_y, b_x, b_y] for c, (t_x, t_y), (b_x, b_y) in boxes]
, device=predictor.device)

In [ ]:
transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, image.shape[:2])
masks, _, _ = predictor.predict_torch(
    point_coords=None,
    point_labels=None,
    boxes=transformed_boxes,
    multimask_output=False,
)

In [ ]:
print(masks.shape)  # (batch_size) x (num_predicted_masks_per_input) x H x W
print(masks[0])
unique_elements = torch.unique(masks[0])
print(unique_elements)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
for mask in masks:
    show_mask(mask.cpu().numpy(), plt.gca(), random_color=True)
for box in input_boxes:
    show_box(box.cpu().numpy(), plt.gca())
plt.axis('off')
plt.show()

## Mask Generator -> Classification

In [ ]:
mask_generator = SamAutomaticMaskGenerator(model=sam,
    points_per_side=16,
    pred_iou_thresh=0.92,
    stability_score_thresh=0.96,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
   )
masks = mask_generator.generate(image)


In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.35)))

In [ ]:
print(masks[0].keys())
masks = [ mask for mask in masks if mask['predicted_iou'] >= 0.95 ]
print(len(masks))
masks = sorted(masks, key=lambda x: x['area'])
masks = masks[::-1]
masks = masks[:8]
for mask in masks:
    print("Area=", mask['area'], )
    print("Stability_score=", mask['stability_score'])
    print("predicted_iou=", mask['predicted_iou'])

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(masks)
plt.axis('off')
plt.show()